# Tensorflow Backdoor Poisoning MNIST Demo 

This demo will cover a basic backdoor poisoning attack on an MNIST-LeNet model.
The following two sections cover experiment setup and is similar across all demos.
To access demo results in MlFlow, please follow the general experiment setup steps outlined in `basic-mlflow-demo`.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_backdoor_poison"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [10]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar czf workflows.tar.gz src/gaussian_augmentation.py src/infer.py src/defenses_image_preprocessing.py src/tasks.py src/attacks_poison_updated.py src/deploy_poison_data.py src/gen_poison_data.py src/train.py src/import_keras.py src/spatial_smoothing.py src/jpeg_compression.py src/init_model.py src/gen_poison_clean_data.py src/gen_poison_model.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [3]:
restapi_client = utils.SecuringAIClient()

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 24,
 'name': 'howard_mnist_backdoor_poison',
 'lastModified': '2021-02-11T09:13:02.448572',
 'createdOn': '2021-02-11T09:13:02.448572'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'name': 'tensorflow_cpu',
  'lastModified': '2020-11-20T17:46:06.756687',
  'createdOn': '2020-11-20T17:46:06.756687',
  'queueId': 1},
 {'name': 'tensorflow_gpu',
  'lastModified': '2020-11-20T18:00:40.876888',
  'createdOn': '2020-11-20T18:00:40.876888',
  'queueId': 2},
 {'name': 'pytorch_cpu',
  'lastModified': '2020-11-20T19:52:36.079781',
  'createdOn': '2020-11-20T19:52:36.079781',
  'queueId': 5},
 {'name': 'pytorch_gpu',
  'lastModified': '2020-11-20T19:52:43.348460',
  'createdOn': '2020-11-20T19:52:43.348460',
  'queueId': 7}]

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [6]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response


# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## MNIST Training: Baseline Model

Next, we need to train our baseline model that will serve as a reference point for the effectiveness of our attacks.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [34]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

response_le_net_train = get_run_id(response_le_net_train)
print_mlflow_results(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T02:43:29.671351',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_backdoor_poison_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 24,
 'jobId': '4e08ad6d-da2f-4e6e-97a6-cf57d06c8a46',
 'lastModified': '2021-05-13T02:43:29.671351',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/b1968fc06b4447e2932b01816f432bf8/workflows.tar.gz'}


KeyboardInterrupt: 

### Generating Poisoned Images

Now we will create our set of poisoned images.
Start by submitting the poison generation job below.

In [67]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar czf workflows.tar.gz src/gaussian_augmentation.py src/infer.py src/defenses_image_preprocessing.py src/tasks.py src/attacks_poison_updated.py src/deploy_poison_data.py src/gen_poison_data.py src/train.py src/import_keras.py src/spatial_smoothing.py src/jpeg_compression.py src/init_model.py src/gen_poison_clean_data.py src/gen_poison_model.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


In [69]:
## Create poisoned test images.
response_gen_poison_le_net_test = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=1",
            "-P label_type=test"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_test)
print("")

response_gen_poison_le_net_test = get_run_id(response_gen_poison_le_net_test)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-05-13T04:07:47.371157',
 'dependsOn': '4e08ad6d-da2f-4e6e-97a6-cf57d06c8a46',
 'entryPoint': 'gen_poison_data',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=100 -P '
                     'target_class=1 -P poison_fraction=1 -P label_type=test',
 'experimentId': 24,
 'jobId': '13faaf6e-e02c-4d80-a908-2f10e7bf2432',
 'lastModified': '2021-05-13T04:07:47.371157',
 'mlflowRunId': None,
 'queueId': 1,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/78f754074238444aa4e1682de2b6f734/workflows.tar.gz'}



In [70]:
## Create poisoned training images.
response_gen_poison_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/training",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=0.1",
            "-P label_type=train"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_train)
print("")

response_gen_poison_le_net_train = get_run_id(response_gen_poison_le_net_train)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-05-13T04:08:39.619121',
 'dependsOn': '4e08ad6d-da2f-4e6e-97a6-cf57d06c8a46',
 'entryPoint': 'gen_poison_data',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/training -P batch_size=100 '
                     '-P target_class=1 -P poison_fraction=0.1 -P '
                     'label_type=train',
 'experimentId': 24,
 'jobId': 'a1b57486-b789-4dfc-ad50-a8f3348a9e77',
 'lastModified': '2021-05-13T04:08:39.619121',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1364d0a81f4b44e78a60ffb550753dd4/workflows.tar.gz'}



In [71]:
# Train new model on poisoned training images
response_le_net_poison_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_data_poison_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
        "-P load_dataset_from_mlruns=true",
        f"-P dataset_run_id_training={response_gen_poison_le_net_train['mlflowRunId']}",
        "-P adv_tar_name=adversarial_poison.tar.gz",
        "-P adv_data_dir=adv_poison_data"
    ]),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train ["jobId"],
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_poison_model)

response_le_net_poison_model = get_run_id(response_le_net_poison_model)    

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T04:17:51.266625',
 'dependsOn': 'a1b57486-b789-4dfc-ad50-a8f3348a9e77',
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id_training=d8f3f95f589345b58eef79bcfcd3006a '
                     '-P adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': '15380ce5-a617-4d7d-9704-b2ca351a4920',
 'lastModified': '2021-05-13T04:17:51.266625',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflo

In [79]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)

print_mlflow_results(response_infer_reg_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T04:46:37.785467',
 'dependsOn': 'a1b57486-b789-4dfc-ad50-a8f3348a9e77',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=d785edaa864e414986bc436ac23bcde1 -P '
                     'model_name=howard_mnist_backdoor_poison_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': '0e6bb254-ecf8-4817-8b3e-b311cc730857',
 'lastModified': '2021-05-13T04:46:37.785467',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/b07e930e00204f49ac8e828df2c8cfeb/workflows.tar.gz'}
{'accuracy': 0.9894999861717224,
 'auc': 0.9995277523994446,
 'loss': 0.03192389262840152,
 'precision': 0.9907824993133545,
 'recall': 0.9889000058174133}


In [80]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_poison_model["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)

print_mlflow_results(response_infer_pos_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T04:48:24.458782',
 'dependsOn': '15380ce5-a617-4d7d-9704-b2ca351a4920',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=d785edaa864e414986bc436ac23bcde1 -P '
                     'model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': 'bdb8c7c1-534b-4fab-9a3c-89f048a15125',
 'lastModified': '2021-05-13T04:48:24.458782',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/30b593f227d14e23a27c9d468558dee1/workflows.tar.gz'}
{'accuracy': 0.11349999904632568,
 'auc': 0.5075914263725281,
 'loss': 20.509925842285156,
 'precision': 0.11349999904632568,
 'recall': 0.11349999904632568}


## Defending against the adversarial backdoor poisoning attack

Now we will explore available defenses on the adversarial backdoor poisoning attack.
The following three jobs will run a selected defense (spatial smoothing, gaussian augmentation, or jpeg compression) and evaluate the defense on the baseline and backdoor trained models.

- The first job uses the selected defense entrypoint to apply a preprocessing defense over the poisoned test images.
- The second job runs the defended images against the poisoned backdoor model.
- The final job runs the defended images against the baseline model.

Ideally the defense will not impact the baseline model accuracy, while improving the backdoor model accuracy scores.

In [82]:
defenses = ["gaussian_augmentation", "spatial_smoothing", "jpeg_compression"]
defense = defenses[1]

response_poison_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point=defense,
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_test["jobId"],
)


print(f"{defense} defense (LeNet architecture) job submitted")
print("")
pprint.pprint(response_poison_def)
print("")

response_poison_def = get_run_id(response_poison_def)

spatial_smoothing defense (LeNet architecture) job submitted

{'createdOn': '2021-05-13T05:39:15.312238',
 'dependsOn': '13faaf6e-e02c-4d80-a908-2f10e7bf2432',
 'entryPoint': 'spatial_smoothing',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=20 -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id=d785edaa864e414986bc436ac23bcde1 -P '
                     'dataset_tar_name=adversarial_poison.tar.gz -P '
                     'dataset_name=adv_poison_data',
 'experimentId': 24,
 'jobId': '80c0f0ca-171b-45ba-918b-0ae4f2e7488e',
 'lastModified': '2021-05-13T05:39:15.312238',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6c504e8643d540b2812a330b11027cbe/workflows.tar.gz'}



In [83]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)
print_mlflow_results(response_infer_pos_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T05:40:52.000626',
 'dependsOn': '80c0f0ca-171b-45ba-918b-0ae4f2e7488e',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=c705e3cae2bd4fdaa064e825aa96e93e -P '
                     'model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 24,
 'jobId': '5d124fb2-0bfb-432f-a50a-fa5fbeefed78',
 'lastModified': '2021-05-13T05:40:52.000626',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/46024827d7864404982ca04f152f7650/workflows.tar.gz'}
{'accuracy': 0.9829000234603882,
 'auc': 0.9996635913848877,
 'loss': 0.05357005931437016,
 'precision': 0.9861041307449341,
 'recall': 0.9793000221252441}


In [84]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)


Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-05-13T05:42:36.064482',
 'dependsOn': '80c0f0ca-171b-45ba-918b-0ae4f2e7488e',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=c705e3cae2bd4fdaa064e825aa96e93e -P '
                     'model_name=howard_mnist_backdoor_poison_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 24,
 'jobId': '7164245f-3dea-4170-a895-6d8f7aed854f',
 'lastModified': '2021-05-13T05:42:36.064482',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/48bcd354bf0f446ba45310f46d76ce7e/workflows.tar.gz'}
{'accuracy': 0.9846000075340271,
 'auc': 0.9994721412658691,
 'loss': 0.046187656186521056,
 'precision': 0.9869372844696045,
 'recall': 0.982200026512146}


<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [ ]:
run_le_net = mlflow_client.get_run(response_le_net_train["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_le_net.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_le_net.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_le_net.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).